Type project: Individual

**Deadline**:11.12.2025 23:59:00

Max possible score: 100

In [1]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from scipy.sparse import hstack
from sklearn.svm import LinearSVC

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
data = pd.read_csv('drive/MyDrive/Aruzhan/project_2/t_zz_text.csv', sep='|', engine='python')
# Убираем лишний столбец с индексом
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

# Убираем пробелы в названиях колонок
data.columns = data.columns.str.strip()

# Проверяем колонки
print(data.columns.tolist())
data.head(10)

['idchain', 'datetimestart', 'datetimestop', 'lentime', 'month_', 'name_1', 'name_2', 'name_3', 'comb_name', 'call_status', 'overall_silence', 'talk_language', 'transcript_operator_words']


,idchain,datetimestart,datetimestop,lentime,month_,name_1,name_2,name_3,comb_name,call_status,overall_silence,talk_language,transcript_operator_words
0,01MMQUNSFK9KDFVKR0F0U2LAES05AASG,2022-12-22 05:01:04,2022-12-22 05:05:13,243.0,12.2022,КРЕДИТЫ,ДЕНЕЖНЫЕ КРЕДИТЫ,ПРОДАЖА \ УСЛОВИЯ ПО НОВОМУ КД,"КРЕДИТЫ, ДЕНЕЖНЫЕ КРЕДИТЫ, ПРОДАЖА \ УСЛОВИЯ П...",NaN,43.850000,kz,здравствуйте сәлеметсіз бе мержан Мұратович қа...
1,01MMQUNSFK9KDFVKR0F0U2LAES05CDPU,2022-12-22 13:59:01,2022-12-22 14:04:34,332.0,12.2022,КАРТА,КРЕДИТНАЯ КАРТА,СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД,"КАРТА, КРЕДИТНАЯ КАРТА, СОСТОЯНИЕ\ УСЛОВИЯ ПО ...",NaN,1.800000,kz,сәлеметсіз бе Хоум кредит банк есімім Ұлан бол...
2,01MMQUNSFK9KDFVKR0F0U2LAES05CDPU,2022-12-22 13:59:01,2022-12-22 14:04:34,332.0,12.2022,КАРТА,КРЕДИТНАЯ КАРТА,СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД,"КАРТА, КРЕДИТНАЯ КАРТА, СОСТОЯНИЕ\ УСЛОВИЯ ПО ...",NaN,5.500000,ru,сәлеметсіз бе здравствуйте Хоум кредит банк ме...
3,01MMQUNSFK9KDFVKR0F0U2LAES05CDPU,2022-12-22 13:59:01,2022-12-22 14:04:34,332.0,12.2022,КАРТА,КРЕДИТНАЯ КАРТА,СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД,"КАРТА, КРЕДИТНАЯ КАРТА, СОСТОЯНИЕ\ УСЛОВИЯ ПО ...",NaN,51.610016,ru,здравствуйте кредит а меня зовут Аслан чем мог...
4,01MMQUNSFK9KDFVKR0F0U2LAES05BEOQ,2022-12-22 09:29:26,2022-12-22 09:30:09,38.0,12.2022,ТРЕТЬЕ ЛИЦО,ИДЕНТИФИКАЦИЯ\ОТКАЗ ОТ ИНФО,ОТКАЗ ОТ ИНФО,"ТРЕТЬЕ ЛИЦО, ИДЕНТИФИКАЦИЯ\ОТКАЗ ОТ ИНФО, ОТКА...",NaN,0.000000,ru,сәлеметсіз бе здравствуйте Хоум кредит банк ес...
5,01MMQUNSFK9KDFVKR0F0U2LAES059U9Q,2022-12-21 13:43:31,2022-12-21 13:46:31,165.0,12.2022,КАРТА,КРЕДИТНАЯ КАРТА,ДРУГОЕ,"КАРТА, КРЕДИТНАЯ КАРТА, ДРУГОЕ",NaN,22.360012,ru,крюке добрый вечер хом кредит банк меня зовут ...
6,01MMQUNSFK9KDFVKR0F0U2LAES059T0V,2022-12-21 13:28:02,2022-12-21 13:29:38,91.0,12.2022,КРЕДИТЫ,ДЕНЕЖНЫЕ КРЕДИТЫ,ПРОДАЖА \ УСЛОВИЯ ПО НОВОМУ КД,"КРЕДИТЫ, ДЕНЕЖНЫЕ КРЕДИТЫ, ПРОДАЖА \ УСЛОВИЯ П...",NaN,17.679985,kz,сәлеметсіз бе қайырлы күн хоум кредит банк есі...
7,01MMQUNSFK9KDFVKR0F0U2LAES05C7D3,2022-12-22 12:53:31,2022-12-22 12:55:42,126.0,12.2022,КРЕДИТЫ,ДЕНЕЖНЫЕ КРЕДИТЫ,СОСТОЯНИЕ\УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД,"КРЕДИТЫ, ДЕНЕЖНЫЕ КРЕДИТЫ, СОСТОЯНИЕ\УСЛОВИЯ П...",NaN,34.890000,kz,сәлеметсіз бе сіз хоум кредит банкке қоңырау ш...
8,01MMQUNSFK9KDFVKR0F0U2LAES05BDE4,2022-12-22 09:16:38,2022-12-22 09:17:26,42.0,12.2022,ПЕРЕКЛЮЧЕНИЯ,ТЕЛЕМАРКЕТИНГ,ТЕЛЕМАРКЕТИНГ,"ПЕРЕКЛЮЧЕНИЯ, ТЕЛЕМАРКЕТИНГ, ТЕЛЕМАРКЕТИНГ",NaN,1.880000,kz,сәлеметсіз бе сіз хоум кредит банкке қоңырау ш...
9,01MMQUNSFK9KDFVKR0F0U2LAES05A06E,2022-12-21 14:15:51,2022-12-21 14:19:12,196.0,12.2022,ИНФО О БАНКЕ,АДРЕСА,ТЕРМИНАЛЫ,"ИНФО О БАНКЕ, АДРЕСА, ТЕРМИНАЛЫ",NaN,107.690020,kz,сәлеметсіз бе есімім Жандос хоум кредит банк қ...


In [11]:
# -------------------------------
# 2. Создание прокси-метки успешности звонка
# -------------------------------
successful_topics = ['КАРТА', 'КРЕДИТЫ']
success_keywords = ['продано', 'оформлено', 'подтверждено', 'закрыт', 'успешно']

def is_successful(row):
    length_silence = (row['lentime'] > 120) and (row['overall_silence'] < 30)
    topic = row['name_1'] in successful_topics
    text = str(row['transcript_operator_words']).lower()
    keyword = any(k in text for k in success_keywords)
    return int(length_silence or topic or keyword)

data['call_success'] = data.apply(is_successful, axis=1)
print(data['call_success'].value_counts())

call_success
1    236738
0    165354
Name: count, dtype: int64


In [12]:
# -------------------------------
# 3. Подготовка данных
# -------------------------------
X_text = data['transcript_operator_words'].astype(str)
X_numeric = data[['lentime', 'overall_silence']]
X_categorical = data[['name_1']]
y = data['call_success']

In [13]:
# Масштабируем числовые признаки
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# One-hot кодируем категориальные признаки
ohe = OneHotEncoder(handle_unknown='ignore')
X_cat_encoded = ohe.fit_transform(X_categorical)

In [14]:
# TF-IDF для текста
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_text_tfidf = tfidf.fit_transform(X_text)

In [15]:
# Объединяем все признаки
X_all = hstack([X_text_tfidf, X_numeric_scaled, X_cat_encoded])

In [16]:
# -------------------------------
# 4. Train/Test split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y, test_size=0.2, random_state=42, stratify=y
)

In [17]:
# -------------------------------
# 5. Модель
# -------------------------------
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Предсказания
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92     33071
           1       0.98      0.90      0.94     47348

    accuracy                           0.93     80419
   macro avg       0.93      0.94      0.93     80419
weighted avg       0.94      0.93      0.93     80419

Accuracy: 0.9315087230629577
